In [1]:
from itertools import combinations
import copy

In [2]:
eps="ε"

In [3]:
in_grammar = """
3
E T F
5
+ * ( ) a
7
E -> E + T
E -> T
T -> T * F
T -> F
F -> a
F -> (E)
E
"""

In [4]:
# in_grammar = """
# 6
# S A B C D E
# 3
# a b c
# 14
# S->A
# S->B
# A->C
# A->D
# B->D
# B->E
# C->S
# C->a
# C->ε
# D->S
# D->b
# E->S
# E->c
# E->ε
# S
# """

In [5]:
grammar = in_grammar.split("\n")
while '' in grammar:
    grammar.remove('')
grammar

['3',
 'E T F',
 '5',
 '+ * ( ) a',
 '7',
 'E -> E + T',
 'E -> T',
 'T -> T * F',
 'T -> F',
 'F -> a',
 'F -> (E)',
 'E']

In [6]:
nonterm = grammar[1].split(" ")
nonterm

['E', 'T', 'F']

In [7]:
term = grammar[3].split(" ")
term

['+', '*', '(', ')', 'a']

In [8]:
num_rules = int(grammar[4])
num_rules

7

In [9]:
in_rules = grammar[5:-1]
in_rules

['E -> E + T', 'E -> T', 'T -> T * F', 'T -> F', 'F -> a', 'F -> (E)']

In [10]:
rules = {}
for rule in in_rules:
    symbol, product = rule.replace(" ", "").split("->")
    if symbol not in rules.keys():
        rules[symbol] = []
    rules[symbol].append(list(product))
rules

{'E': [['E', '+', 'T'], ['T']],
 'T': [['T', '*', 'F'], ['F']],
 'F': [['a'], ['(', 'E', ')']]}

In [11]:
original_rules = copy.deepcopy(rules)

In [12]:
start_symb = grammar[-1]
start_symb

'E'

# Эпсилон правила

In [13]:
leads_to_eps = set()
for k, v in rules.items():
    if [eps] in v:
        leads_to_eps.add(k)
leads_to_eps

set()

In [14]:
flag = True
while flag:
    flag = False
    for k, v in rules.items():
        for i in v:
            if all(x in leads_to_eps for x in i) and k not in leads_to_eps:
                flag = True
                leads_to_eps.add(k)
leads_to_eps

set()

In [15]:
new_rules = copy.deepcopy(rules)
new_rules

{'E': [['E', '+', 'T'], ['T']],
 'T': [['T', '*', 'F'], ['F']],
 'F': [['a'], ['(', 'E', ')']]}

In [16]:
def remove_all_from_list(a, b):
    return [x for x in a if x not in b]

In [17]:
for k, v in rules.items():
    for i in v:
        leads_to_eps_in_curr = [x for x in i if x in leads_to_eps]
        if leads_to_eps_in_curr:
            for len_perm in range(1, len(leads_to_eps_in_curr)+1):
                for j in combinations(leads_to_eps_in_curr, len_perm):
                    to_append = remove_all_from_list(i, j)
                    if to_append:
                        new_rules[k].append(remove_all_from_list(i, j))
            


In [18]:
new_rules

{'E': [['E', '+', 'T'], ['T']],
 'T': [['T', '*', 'F'], ['F']],
 'F': [['a'], ['(', 'E', ')']]}

In [19]:
for k, v in new_rules.items():
    if [eps] in v:
        v.remove([eps])
new_rules

{'E': [['E', '+', 'T'], ['T']],
 'T': [['T', '*', 'F'], ['F']],
 'F': [['a'], ['(', 'E', ')']]}

In [20]:
rules

{'E': [['E', '+', 'T'], ['T']],
 'T': [['T', '*', 'F'], ['F']],
 'F': [['a'], ['(', 'E', ')']]}

In [21]:
if start_symb in leads_to_eps:
    new_start_symbol = start_symb+"*"
    new_rules[new_start_symbol] = [[eps], [start_symb]]
    start_symb = new_start_symbol
new_rules

{'E': [['E', '+', 'T'], ['T']],
 'T': [['T', '*', 'F'], ['F']],
 'F': [['a'], ['(', 'E', ')']]}

In [22]:
rules = copy.deepcopy(new_rules) # Теперь без eps
rules

{'E': [['E', '+', 'T'], ['T']],
 'T': [['T', '*', 'F'], ['F']],
 'F': [['a'], ['(', 'E', ')']]}

# Цепные правила

In [23]:
N_dict = {}
for k in new_rules.keys():
    N = set()
    N.add(k)
    N0 = copy.deepcopy(N)
    while True:
        for symb in N0:
            for rule in rules[symb]:
                if (len(rule)==1) and (rule[0] in rules.keys()):
                    N.add(rule[0])
        if N==N0:
            break
        N0 = copy.deepcopy(N)
    N_dict[k] = list(N)
N_dict


{'E': ['F', 'E', 'T'], 'T': ['F', 'T'], 'F': ['F']}

In [24]:
new_rules = {}
for k in rules.keys():
    new_rules[k] = []
    for i in N_dict[k]:
        for rule in rules[i]:
            if (len(rule)==1) and (rule[0] in rules.keys()):
                continue
            new_rules[k].append(rule)
new_rules

{'E': [['a'], ['(', 'E', ')'], ['E', '+', 'T'], ['T', '*', 'F']],
 'T': [['a'], ['(', 'E', ')'], ['T', '*', 'F']],
 'F': [['a'], ['(', 'E', ')']]}

In [25]:
rules = copy.deepcopy(new_rules) # Теперь без цепных правил

# Непосредственная левая рекурсия

In [26]:
def remove_direct_lr(symb, res):
    new_symb = symb+"'"
    beta_type = [x for x in res if x[0]!=symb]
    alpha_type = [x[1:] for x in res if x not in beta_type]
    new_rules_for_symb = [x+[new_symb] for x in beta_type] + beta_type
    rules_for_new_symb = [x+[new_symb] for x in alpha_type] + alpha_type
    return new_rules_for_symb, new_symb, rules_for_new_symb

In [27]:
remove_direct_lr("E", rules["E"])

([['a', "E'"],
  ['(', 'E', ')', "E'"],
  ['T', '*', 'F', "E'"],
  ['a'],
  ['(', 'E', ')'],
  ['T', '*', 'F']],
 "E'",
 [['+', 'T', "E'"], ['+', 'T']])

In [28]:
rules

{'E': [['a'], ['(', 'E', ')'], ['E', '+', 'T'], ['T', '*', 'F']],
 'T': [['a'], ['(', 'E', ')'], ['T', '*', 'F']],
 'F': [['a'], ['(', 'E', ')']]}

In [29]:
new_rules = copy.deepcopy(rules)

# Левая рекурсия

In [30]:
for i in range(len(nonterm)):
    for j in range(i):
        for rule_to_check in rules[nonterm[i]]:
            if rule_to_check[0]==nonterm[j]:
                new_rules[nonterm[i]].remove(rule_to_check)
                for extra_rule in new_rules[nonterm[j]]:
                    new_rules[nonterm[i]].append(extra_rule + rule_to_check[1:])
    a, b, c = remove_direct_lr(nonterm[i], new_rules[nonterm[i]])
    if c:
        new_rules[nonterm[i]] = a

        new_rules[b] = c
new_rules

{'E': [['a', "E'"],
  ['(', 'E', ')', "E'"],
  ['T', '*', 'F', "E'"],
  ['a'],
  ['(', 'E', ')'],
  ['T', '*', 'F']],
 'T': [['a', "T'"], ['(', 'E', ')', "T'"], ['a'], ['(', 'E', ')']],
 'F': [['a'], ['(', 'E', ')']],
 "E'": [['+', 'T', "E'"], ['+', 'T']],
 "T'": [['*', 'F', "T'"], ['*', 'F']]}

In [31]:
rules = copy.deepcopy(new_rules)

# Бесполезные символы

In [32]:
# 2.7
N = set()
flag = True
while flag:
    flag = False
    for k, v in rules.items():
        for rule in v:
            if all(x for x in nonterm+list(N)) and k not in N:
                N.add(k)
                flag = True


In [33]:
N_e = copy.deepcopy(N)
N_e

{'E', "E'", 'F', 'T', "T'"}

In [34]:
if start_symb in N_e:
    print("YES")
else:
    print("NO")

YES


In [35]:
term

['+', '*', '(', ')', 'a']

In [36]:
term + list(N_e)

['+', '*', '(', ')', 'a', "E'", 'F', 'E', 'T', "T'"]

In [37]:
rules

{'E': [['a', "E'"],
  ['(', 'E', ')', "E'"],
  ['T', '*', 'F', "E'"],
  ['a'],
  ['(', 'E', ')'],
  ['T', '*', 'F']],
 'T': [['a', "T'"], ['(', 'E', ')', "T'"], ['a'], ['(', 'E', ')']],
 'F': [['a'], ['(', 'E', ')']],
 "E'": [['+', 'T', "E'"], ['+', 'T']],
 "T'": [['*', 'F', "T'"], ['*', 'F']]}

In [38]:
new_rules = {}
for k, v in rules.items():
    if k in N_e and k not in new_rules.keys():
        new_rules[k] = []
    for rule in v:
        if all(x in term + list(N_e) +[eps] for x in rule):
            new_rules[k].append(rule)
new_rules

{'E': [['a', "E'"],
  ['(', 'E', ')', "E'"],
  ['T', '*', 'F', "E'"],
  ['a'],
  ['(', 'E', ')'],
  ['T', '*', 'F']],
 'T': [['a', "T'"], ['(', 'E', ')', "T'"], ['a'], ['(', 'E', ')']],
 'F': [['a'], ['(', 'E', ')']],
 "E'": [['+', 'T', "E'"], ['+', 'T']],
 "T'": [['*', 'F', "T'"], ['*', 'F']]}

In [39]:
nonterm = list(new_rules.keys())
nonterm

['E', 'T', 'F', "E'", "T'"]

In [40]:
rules = copy.deepcopy(new_rules)

In [41]:
# 2.8
V = set()
V.add(start_symb)
while True:
    V0 = copy.deepcopy(V)
    for i in V0:
        if i not in rules.keys():
            continue
        for rule in rules[i]:
            for a in rule:
                V.add(a)

    if V0==V:
        break

In [42]:
V

{'(', ')', '*', '+', 'E', "E'", 'F', 'T', "T'", 'a'}

In [43]:
nonterm = [x for x in rules.keys() if x in V]
term = [x for x in term if x in V]

In [44]:
term

['+', '*', '(', ')', 'a']

In [45]:
new_rules = {}
for k, v in rules.items():
    if k not in nonterm:
        continue
    new_rules[k] = []
    for rule in v:
        if all(x in term + nonterm + [eps] for x in rule):
            new_rules[k].append(rule)
new_rules

{'E': [['a', "E'"],
  ['(', 'E', ')', "E'"],
  ['T', '*', 'F', "E'"],
  ['a'],
  ['(', 'E', ')'],
  ['T', '*', 'F']],
 'T': [['a', "T'"], ['(', 'E', ')', "T'"], ['a'], ['(', 'E', ')']],
 'F': [['a'], ['(', 'E', ')']],
 "E'": [['+', 'T', "E'"], ['+', 'T']],
 "T'": [['*', 'F', "T'"], ['*', 'F']]}